In [1]:
import logging
logging.basicConfig(level=logging.DEBUG)
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from ot_dis.wtk.utilities import get_ucr_dataset, krein_svm_grid_search
from ot_dis.wtk import transform_to_dist_matrix, get_kernel_matrix, wtk_distance
import numpy as np
import os
from getData import getData
from knn import knn_OTW

DEBUG:matplotlib:matplotlib data path: c:\Users\Administrator\Documents\WTK-distance\venv\Lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\Administrator\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\Administrator\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\Administrator\.matplotlib\fontlist-v330.json


In [2]:
import torch
import numpy as np
from getData import getData
import seaborn as sns
import os
from ot_dis.kpg import kpg_sequence_distance, kpg_sequence_partial_distance
from knn import knn_sequence_KPG
from ot_dis.wtk import wtk_distance
from arabic_load import get_train_data, get_test_data

# 1. Load data

In [3]:
X_train, y_train, X_test, y_test = getData('BME', '../data/UCR/raw_data/')
len(X_train), len(X_test)

(30, 150)

In [4]:
len(X_train), len(X_test), X_train[0].shape

(30, 150, (128,))

In [9]:
X_train_arabic, y_train_arabic = get_train_data()
X_test_arabic, y_test_arabic = get_test_data()
len(X_train_arabic), len(X_test_arabic)

(6600, 2200)

In [10]:
X_train_arabic = X_train_arabic[:660]
X_test_arabic = X_test_arabic[:100]
y_train_arabic = y_train_arabic[:660]
y_test_arabic = y_test_arabic[:100]

In [11]:
X_train_arabic[0].shape

(38, 13)

# 2. Training

## 2.1. Sequence KPG

### 2.1.1. 2D data

In [12]:
lamb = 5
accuracy = knn_sequence_KPG(X_train=X_train_arabic, X_test=X_test_arabic, y_train=y_train_arabic, y_test=y_test_arabic,  method="2d_normal",lamb=lamb)
accuracy

  0%|          | 0/660 [00:00<?, ?it/s]c:\Users\Administrator\Documents\WTK-distance\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 660/660 [10:52<00:00,  1.01it/s]


1.0

### 2.1.2. 1D data with sub sequence

In [13]:
sub_length = 64
lamb = 45
accuracy = knn_sequence_KPG(X_train=X_train, X_test=X_test, y_train=y_train, y_test=y_test,  method="1d_sequence",lamb=lamb, sub_length=sub_length)
accuracy

  0%|          | 0/30 [00:00<?, ?it/s]c:\Users\Administrator\Documents\WTK-distance\venv\Lib\site-packages\cvxpy\reductions\solvers\solving_chain.py:336: FutureWarning: 
    Your problem is being solved with the ECOS solver by default. Starting in 
    CVXPY 1.5.0, Clarabel will be used as the default solver instead. To continue 
    using ECOS, specify the ECOS solver explicitly using the ``solver=cp.ECOS`` 
    argument to the ``problem.solve`` method.
    
  warnings.warn(ECOS_DEPRECATION_MSG, FutureWarning)
100%|██████████| 30/30 [03:02<00:00,  6.08s/it]


0.26

## 2.2. Partial KPG

### 2.2.1. 1D data with sub sequence

### 2.2.2. 1D data